In [3]:
from kafka import KafkaProducer
from time import sleep
from json import dumps
import numpy as np
np.random.seed(41)

from kafka import KafkaConsumer
from json import loads

#for fashion
import pandas as pd
from fashion_mnist_prediction import prediction

In [2]:
#creating topic
!/Applications/kafka_2.13-3.0.0/bin/kafka-topics.sh --create --topic prediction_response --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1

Created topic prediction_response.


In [5]:
## read test data
test_df = pd.read_csv('data/test_data.csv')
y_test = test_df['Label'].values
x_test = test_df.drop('Label', axis=1)
x_test = np.reshape(x_test.values, (len(x_test), 28, 28))
#initialize the producer
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                        value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

class_name = {0 : 'T-shirt/top', 1:'Trouser', 2:'Pullover', 3:'Dress', 4:'Coat',5:'Sandal', 6:'Shirt', 
             7: 'Sneaker', 8: 'Bag', 9:'Ankle Boot'}

In [ ]:
#consumer
consumer = KafkaConsumer('request',
                         bootstrap_servers=['localhost:9092'],
                        value_deserializer=lambda x: loads(x.decode('utf-8')))
for message in consumer:
    image_idx = int(message.value)
    if image_idx != None:
        pred_result = class_name[prediction(x_test[image_idx], model_path='models/Resnet-18-epoch-2.pth')]
        producer.send('prediction_response', value = 'Prediction Result: ' + pred_result)